In [1]:
import os, sys

import time
start_time = time.time()

import io
import re
import json
import boto3
import pickle
import stanza
import logging
import pandas as pd
from pymongo import MongoClient

In [2]:
os.environ["bucket"] = 'frus-corenlp'

# Configure s3 client and resource
logging.critical("Configure s3...")
bucket = os.environ["bucket"]
# s3 = boto3.client('s3', aws_access_key_id=aws_id, aws_secret_access_key=aws_secret_key)
# s3_resource = boto3.resource('s3', aws_access_key_id=aws_id, aws_secret_access_key=aws_secret_key)

s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

CRITICAL:root:Configure s3...


In [3]:
# stanza.download('en', '.') # download English model

In [4]:
# stanza.install_corenlp()

In [5]:
# nlp = stanza.Pipeline('en', '.')

In [23]:
def func(a):
    print(a)
    return 1

In [24]:
def get_batch(b):
    key = "dataframes/batch-{:04d}.csv".format(b)
    data = s3_resource.Object("frus-corenlp", key).get()['Body'].read()
    df = pd.read_csv(io.BytesIO(data)).iloc[:, 1:]
    return df

In [38]:
from itertools import count
from multiprocess import Process, Pool

import multiprocessing as mp
import handler

num_workers = mp.cpu_count() 
print(num_workers)

8


In [39]:
timeout = 120
context = {}
was = {}
to_parse = {}

num_threads = 2 #mp.cpu_count() 
ps = [None] * num_threads
results = [None] * num_threads
for b in range(2, len(ps)+2):
    key = "dataframes/batch-{:04d}.csv".format(b)
    event = {"key": key, "timeout":timeout}

    ps[b-2] = Process(target=handler.handler, args=(event,))
    ps[b-2].start()
    was[b] = 'all'
    to_parse[b] = 6466

last_time = time.time()

# for b in range(len(ps)):
#     ps[b].join(timeout=timeout+10)
ps

[<Process name='Process-17' pid=20148 parent=7952 started>,
 <Process name='Process-18' pid=13428 parent=7952 started>]

In [40]:
ps

[<Process name='Process-17' pid=20148 parent=7952 started>,
 <Process name='Process-18' pid=13428 parent=7952 started>]

In [20]:
[print(ps[i]) for i in range(2, len(ps)+2)]

print("\nIt has been {:.2f} minutes since invoking the Lambda functions.".format((time.time() - last_time) / 60))
for b in range(2, len(ps)+2):
    df = get_batch(b)
    if 'parsed' not in df.columns:
        df['parsed'] = None
    print('Batch {0:4,}:  {1:3} ({2:3} last) {3:3}'.format(b, df.parsed.isnull().sum(), was[b], to_parse[b]))
    was[b] = df.parsed.isnull().sum()

[<Process name='Process-7' pid=18752 parent=7952 started>, <Process name='Process-8' pid=18840 parent=7952 started>]

It has been 0.74 minutes since invoking the Lambda functions.
Batch    2:  6466 (all last) 6466
Batch    3:  6466 (all last) 6466


In [21]:
get_batch(2)

,id,id.1,text,title,url,date,website,language,decade,num_words,num_capitalized_words,perc_capitalized_words,batch
0,2,2,The Consul General at Salisbury ( Sims ) to th...,[3] The Consul General at Salisbury ( Sims ) t...,3,1952-03-18,Truman,English,1950,2530,306,0.120949,2
1,52,52,"Memorandum of Conversation, by the Director, O...","[193] Memorandum of Conversation, by the Direc...",55,1953-03-24,Truman,English,1950,1121,171,0.152542,2
2,102,102,"Memorandum of Conversation, by Alexander J. Da...","[246] Memorandum of Conversation, by Alexander...",105,1954-10-29,Truman,English,1950,1327,247,0.186134,2
3,152,152,The Secretary of State to the Legation in Liby...,[299] The Secretary of State to the Legation i...,155,1954-08-24,Truman,English,1950,556,61,0.109712,2
4,202,202,The Secretary of State to the United States Mi...,[357] The Secretary of State to the United Sta...,205,1954-12-11,Truman,English,1950,311,46,0.147910,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6461,323052,323052,104. Memorandum From Secretary of State Rogers...,104. Memorandum From Secretary of State Rogers...,178055,1970-11-24,Nixon-Ford,English,1970,926,151,0.163067,2
6462,323102,323102,250. Memorandum From A. Denis Clift of the Nat...,250. Memorandum From A. Denis Clift of the Nat...,178555,1972-08-28,Nixon-Ford,English,1970,695,132,0.189928,2
6463,323152,323152,33. Memorandum From Acting Secretary of State ...,33. Memorandum From Acting Secretary of State ...,321918,1969-08-16,Nixon-Ford,English,1960,963,180,0.186916,2
6464,323202,323202,"87. Memorandum From the Assistant Director, Ne...","87. Memorandum From the Assistant Director, Ne...",84359,1970-01-01,Nixon-Ford,English,1970,641,83,0.129485,2


In [ ]:
num_threads = mp.cpu_count()
timeout = 300
data = [{"key": "dataframes/batch-{:04d}.csv".format(b), "timeout":timeout} for b in range(2, num_threads+2)]
print(data)

pool = Pool(num_threads)
results = pool.map(handler.handler, data)
results

[{'key': 'dataframes/batch-0002.csv', 'timeout': 300}, {'key': 'dataframes/batch-0003.csv', 'timeout': 300}, {'key': 'dataframes/batch-0004.csv', 'timeout': 300}, {'key': 'dataframes/batch-0005.csv', 'timeout': 300}, {'key': 'dataframes/batch-0006.csv', 'timeout': 300}, {'key': 'dataframes/batch-0007.csv', 'timeout': 300}, {'key': 'dataframes/batch-0008.csv', 'timeout': 300}, {'key': 'dataframes/batch-0009.csv', 'timeout': 300}]


In [42]:
print("It has been {:.2f} minutes since invoking the Lambda functions.".format((time.time() - last_time) / 60))
for b in range(2, len(ps)+2):
    df = get_batch(b)
    if 'parsed' not in df.columns:
        df['parsed'] = None
    print('Batch {0:4,}:  {1:3} ({2:3} last) {3:3}'.format(b, df.parsed.isnull().sum(), was[b], to_parse[b]))
    was[b] = df.parsed.isnull().sum()

It has been 0.65 minutes since invoking the Lambda functions.
Batch    2:  6452 (6452 last) all
Batch    3:  6453 (6453 last) all


In [84]:
results

[1, 1]